# Unit 5: People Perception Part 2. Face Recognition

You now know how to detect different faces, but what if you want to recognise a familiar face? Well, this is what you will learn here.

You will work with this scene: a Fetch robot and three different people that you will have to be able to recognise.

<img src="img/perception_unit5_threepersons.png"/>

## Start The Face Recognition Package

For this, you will use the <a href="https://github.com/ageitgey/face_recognition">face_recognition python module</a>.

It has many features apart from the FaceRecognition feature, like face finding or applying digital makeup. But it gives a very simple way of doing a rudimentary face recognition. Here you have an example of how to recognise **one face**:

There are some elements to comment on:

In [ ]:
import cv2
import face_recognition
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
import rospkg

You will need these special packages, like **cv2** (OpenCV), **cv_bridge** (CV_Bridge), and **face_recognition** to be able to recognize the faces. You will also need **rospkg** in order to easily find files that are inside other ROS packages.

In [ ]:
# get an instance of RosPack with the default search paths
rospack = rospkg.RosPack()
# get the file path for face_recognition_pkg
self.path_to_package = rospack.get_path('face_recognition_pkg')
print self.path_to_package

Then, you need to **retrieve an image** of the person that you want to recognise. **It is important to note** that by giving only one image, the face recognition system won't make positive detections unless what it visualizes is very similar to the image provided. So, the more images you provide using different configurations, the better the detections will be. Anyways, one image is more than enough.

In [ ]:
# Load a sample picture and learn how to recognize it.
image_path = os.path.join(self.path_to_package,"scripts/standing_person.png")

<img src="img/perception_unit5_standing_person2.png"/>

Once you have that, it starts the real image processing:

In [ ]:
standing_person_image = face_recognition.load_image_file(image_path)
standing_person_face_encoding = face_recognition.face_encodings(standing_person_image)[0]        

Here you see that you load the image into a **face_recognition** variable, and then you extract the encodings. Note that you are getting the first encoding [0]. This is because you only have one image of that person. More images will generate more encodings.<br>

We initialise some variables and then we underscale the image retrieved from the cameras. In this case, half of the original size. That way, you work with less pixels, making the recognition process faster. But, when you show the images captured, you will show the original size.<br>
But this has a side effect: if you reduce the size **too much,** the face encoder has less information to work with and, therefore, it makes it harder to extract human features and match them with someone.<br>
It has to be a compromise between performance and functionality. If you are too far away from your subjects, then you will have to work with bigger images.

<img src="img/perception_unit5_smallbobeye.png"/>

With an image this small, it will be difficult for the algorithm to work, so you will have to get closer.

In [ ]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

# Resize frame of video to 1/2 size for faster face recognition processing
small_frame = cv2.resize(video_capture, (0, 0), fx=0.5, fy=0.5)

Let's process the image frame. Bear in mind that, here, you are processing one and not the next one. This, again, is to reduce the number of images to be processed, making the recognition faster.

In [ ]:
# Only process every other frame of video to save time
if process_this_frame:
    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(small_frame)
    face_encodings = face_recognition.face_encodings(small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces([standing_person_face_encoding], face_encoding)
        name = "Unknown"

        if match[0]:
            print "MATCH"
            name = "StandingPerson"

        face_names.append(name)

process_this_frame = not process_this_frame

First, extract the location of the faces. Then, from those locations, you extract the encodings. These encodings are like the "fingerprint" of each face, defining it. You do this for each image frame that you capture.<br>
Then, for all the faces detected and, therefore, encodings, you **compare** those encodings with the standing_person_face_encoding. If there is a match, then you found the standing person, and you add that name to the face_names list.

Finally, you display the results of the face recognition. This means showing the image captured and the locations where you detected a face, and draw a square with the name of the recognised face there.

In [ ]:
# Display the results
for (top, right, bottom, left), name in zip(face_locations, face_names):
    # Scale back up face locations since the frame we detected in was scaled to 1/2 size
    top *= 2
    right *= 2
    bottom *= 2
    left *= 2

    # Draw a box around the face
    cv2.rectangle(video_capture, (left, top), (right, bottom), (0, 0, 255), 2)

    # Draw a label with a name below the face
    cv2.rectangle(video_capture, (left, bottom - 35), (right, bottom), (0, 0, 255))
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(video_capture, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

# Display the resulting image
cv2.imshow("Image window", video_capture)
cv2.waitKey(1)

The result of this should be the following:

<img src="img/perception_unit5_threepeopleunknown.png"/>

And this is the final code you would have, including all the parts explained:

<p style="background:green;color:white;">**recognise_face.py**</p>

In [ ]:
#!/usr/bin/env python
import os
import rospy
import cv2
import face_recognition
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
import rospkg



class FaceRecogniser(object):

    def __init__(self):
        rospy.loginfo("Start FaceRecogniser Init process...")
        # get an instance of RosPack with the default search paths
        rospack = rospkg.RosPack()
        # get the file path for face_recognition_pkg
        self.path_to_package = rospack.get_path('face_recognition_pkg')

        self.bridge_object = CvBridge()
        rospy.loginfo("Start camera suscriber...")
        self.image_sub = rospy.Subscriber("/head_camera/rgb/image_raw",Image,self.camera_callback)
        rospy.loginfo("Finished FaceRecogniser Init process...Ready")
        
    def camera_callback(self,data):
        
        self.recognise(data)

    def recognise(self,data):
        
        # Get a reference to webcam #0 (the default one)
        try:
            # We select bgr8 because its the OpneCV encoding by default
            video_capture = self.bridge_object.imgmsg_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)
        
        
        # Load a sample picture and learn how to recognize it.
        image_path = os.path.join(self.path_to_package,"person_img/standing_person.png")
        
        standing_person_image = face_recognition.load_image_file(image_path)
        standing_person_face_encoding = face_recognition.face_encodings(standing_person_image)[0]
        
        
        # Initialize some variables
        face_locations = []
        face_encodings = []
        face_names = []
        process_this_frame = True

        # Resize frame of video to 1/2 size for faster face recognition processing
        # If this is done be aware that you will have to make the recognition nearer.
        # In this case it will work around maximum 1 meter, more it wont work properly
        small_frame = cv2.resize(video_capture, (0, 0), fx=0.5, fy=0.5)
        
        #cv2.imshow("SMALL Image window", small_frame)
        
        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(small_frame)
            face_encodings = face_recognition.face_encodings(small_frame, face_locations)
    
            if not face_encodings:
                rospy.logwarn("No Faces found, please get closer...")
                
            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                match = face_recognition.compare_faces([standing_person_face_encoding], face_encoding)
                name = "Unknown"
    
                if match[0]:
                    rospy.loginfo("MATCH")
                    name = "StandingPerson"
                else:
                    rospy.logwarn("NO Match")
    
                face_names.append(name)

        process_this_frame = not process_this_frame
    
        for (top, right, bottom, left), name in zip(face_locations, face_names):

            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 2
            right *= 2
            bottom *= 2
            left *= 2
    
            # Draw a box around the face
            cv2.rectangle(video_capture, (left, top), (right, bottom), (0, 0, 255), 2)
    
            # Draw a label with a name below the face
            cv2.rectangle(video_capture, (left, bottom - 35), (right, bottom), (0, 0, 255))
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(video_capture, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow("Image window", video_capture)
        cv2.waitKey(1)
        



def main():
    rospy.init_node('face_recognising_python_node', anonymous=True)
   
    line_follower_object = FaceRecogniser()

    rospy.spin()
    cv2.destroyAllWindows()

    
if __name__ == '__main__':
    main()

<p style="background:green;color:white;">**END recognise_face.py**</p>

Let's create a package called **my_face_recogniser**:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd;cd ../src
catkin_create_pkg my_face_recogniser rospy sensor_msgs cv_bridge

Add to your package some example recognition images. You can get this images from the **person_img** folder of the <i>face_recognition_pkg</i> package. To copy this folder to your package, just execute the following commands:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd face_recognition_pkg/
cp -r person_img /home/user/catkin_ws/src/my_face_recogniser

You will also need a script to move Fetch robot closer to the people. Here you can see an example script:

<p style="background:green;color:white;">**move_fetch_robot_client.py**</p>

In [ ]:
#!/usr/bin/env python

# Copyright (c) 2015, Fetch Robotics Inc.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#     * Neither the name of the Fetch Robotics Inc. nor the names of its
#       contributors may be used to endorse or promote products derived from
#       this software without specific prior written permission.
# 
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL FETCH ROBOTICS INC. BE LIABLE FOR ANY
# DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
# (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
# LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
# ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF
# THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# Author: Michael Ferguson
import numpy
import copy
import actionlib
import rospy
import time
from math import sin, cos
from moveit_python import (MoveGroupInterface,
                           PlanningSceneInterface,
                           PickPlaceInterface)
from moveit_python.geometry import rotate_pose_msg_by_euler_angles

from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from control_msgs.msg import PointHeadAction, PointHeadGoal
from geometry_msgs.msg import Twist, Pose
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from moveit_msgs.msg import PlaceLocation, MoveItErrorCodes
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from nav_msgs.msg import Odometry

# Move base using navigation stack
class MoveBasePublisher(object):

    def __init__(self):
        self._move_base_publisher = rospy.Publisher('/base_controller/command', Twist, queue_size=1)
        self.odom_subs = rospy.Subscriber("/odom", Odometry, self.odom_callback)
        self.actual_pose = Pose()
    
    def odom_callback(self,msg):
        self.actual_pose = msg.pose.pose

    def move_to(self, twist_object):
        self._move_base_publisher.publish(twist_object)
    
    def stop(self):
        twist_object = Twist()
        self.move_to(twist_object)
        
    def move_forwards_backwards(self,speed_ms,position_x, allowed_error=0.1):
        """
        distance_metres: positive is go forwards, negative go backwards
        """
        rate = rospy.Rate(5)
        move_twist = Twist()
        
        start_x = self.actual_pose.position.x
        direction_speed = numpy.sign(position_x - start_x)
        move_twist.linear.x = direction_speed*speed_ms
        
        
        in_place = False
        while not in_place:
            self.move_to(move_twist)
            x_actual = self.actual_pose.position.x
            print x_actual

            if  abs(position_x - x_actual) <= allowed_error:
                print "Reached position"
                break

            rate.sleep()
        self.stop()


# Move base using navigation stack
class MoveBaseClient(object):

    def __init__(self):
        self.client = actionlib.SimpleActionClient("move_base", MoveBaseAction)
        rospy.loginfo("Waiting for move_base...")
        self.client.wait_for_server()

    def goto(self, x, y, theta, frame="map"):
        move_goal = MoveBaseGoal()
        move_goal.target_pose.pose.position.x = x
        move_goal.target_pose.pose.position.y = y
        move_goal.target_pose.pose.orientation.z = sin(theta/2.0)
        move_goal.target_pose.pose.orientation.w = cos(theta/2.0)
        move_goal.target_pose.header.frame_id = frame
        move_goal.target_pose.header.stamp = rospy.Time.now()

        # TODO wait for things to work
        self.client.send_goal(move_goal)
        self.client.wait_for_result()

# Send a trajectory to controller
class FollowTrajectoryClient(object):

    def __init__(self, name, joint_names):
        self.client = actionlib.SimpleActionClient("%s/follow_joint_trajectory" % name,
                                                   FollowJointTrajectoryAction)
        rospy.loginfo("Waiting for %s..." % name)
        self.client.wait_for_server()
        self.joint_names = joint_names

    def move_to(self, positions, duration=5.0):
        if len(self.joint_names) != len(positions):
            print("Invalid trajectory position")
            return False
        trajectory = JointTrajectory()
        trajectory.joint_names = self.joint_names
        trajectory.points.append(JointTrajectoryPoint())
        trajectory.points[0].positions = positions
        trajectory.points[0].velocities = [0.0 for _ in positions]
        trajectory.points[0].accelerations = [0.0 for _ in positions]
        trajectory.points[0].time_from_start = rospy.Duration(duration)
        follow_goal = FollowJointTrajectoryGoal()
        follow_goal.trajectory = trajectory

        self.client.send_goal(follow_goal)
        self.client.wait_for_result()

# Point the head using controller
class PointHeadClient(object):

    def __init__(self):
        self.client = actionlib.SimpleActionClient("head_controller/point_head", PointHeadAction)
        rospy.loginfo("Waiting for head_controller...")
        self.client.wait_for_server()

    def look_at(self, x, y, z, frame, duration=1.0):
        goal = PointHeadGoal()
        goal.target.header.stamp = rospy.Time.now()
        goal.target.header.frame_id = frame
        goal.target.point.x = x
        goal.target.point.y = y
        goal.target.point.z = z
        goal.min_duration = rospy.Duration(duration)
        self.client.send_goal(goal)
        self.client.wait_for_result()

# Tools for grasping
class GraspingClient(object):

    def __init__(self):
        self.scene = PlanningSceneInterface("base_link")
        self.pickplace = PickPlaceInterface("arm", "gripper", verbose=True)
        self.move_group = MoveGroupInterface("arm", "base_link")

   
    def getGraspableCube(self):
        
        # nothing detected
        return None, None

    def getSupportSurface(self, name):
        for surface in self.support_surfaces:
            if surface.name == name:
                return surface
        return None

    def getPlaceLocation(self):
        pass

    def pick(self, block, grasps):
        success, pick_result = self.pickplace.pick_with_retry(block.name,
                                                              grasps,
                                                              support_name=block.support_surface,
                                                              scene=self.scene)
        self.pick_result = pick_result
        return success

    def place(self, block, pose_stamped):
        places = list()
        l = PlaceLocation()
        l.place_pose.pose = pose_stamped.pose
        l.place_pose.header.frame_id = pose_stamped.header.frame_id

        # copy the posture, approach and retreat from the grasp used
        l.post_place_posture = self.pick_result.grasp.pre_grasp_posture
        l.pre_place_approach = self.pick_result.grasp.pre_grasp_approach
        l.post_place_retreat = self.pick_result.grasp.post_grasp_retreat
        places.append(copy.deepcopy(l))
        # create another several places, rotate each by 90 degrees in yaw direction
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))

        success, place_result = self.pickplace.place_with_retry(block.name,
                                                                places,
                                                                scene=self.scene)
        return success

    def tuck(self):
        joints = ["shoulder_pan_joint", "shoulder_lift_joint", "upperarm_roll_joint",
                  "elbow_flex_joint", "forearm_roll_joint", "wrist_flex_joint", "wrist_roll_joint"]
        pose = [1.32, 1.40, -0.2, 1.72, 0.0, 1.66, 0.0]
        while not rospy.is_shutdown():
            result = self.move_group.moveToJointPosition(joints, pose, 0.02)
            if result.error_code.val == MoveItErrorCodes.SUCCESS:
                return



def Move_to_Two_locations_demo():
    # Create a node
    rospy.init_node("move_to_two_locations_demo_node")

    # Make sure sim time is working
    while not rospy.Time.now():
        pass

    # Setup clients
    move_base = MoveBaseClient()
    torso_action = FollowTrajectoryClient("torso_controller", ["torso_lift_joint"])
    head_action = PointHeadClient()
    grasping_client = GraspingClient()
    
    rospy.loginfo("Start Sequence complete")
    
    
    # Move the base to be in front of the table
    # Demonstrates the use of the navigation stack
    rospy.loginfo("Moving to table...")
    move_base.goto(2.250, 3.118, 0.0)
    move_base.goto(2.750, 3.118, 0.0)
    
    # Raise the torso using just a controller
    rospy.loginfo("Raising torso...")
    torso_action.move_to([0.4, ])

    # Point the head at the cube we want to pick
    head_action.look_at(3.7, 3.18, 0.0, "map")

    # Tuck the arm
    grasping_client.tuck()

    # Lower torso
    rospy.loginfo("Lowering torso...")
    torso_action.move_to([0.0, ])

    # Move to second table
    rospy.loginfo("Moving to second table...")
    move_base.goto(-3.53, 3.75, 1.57)
    move_base.goto(-3.53, 4.15, 1.57)

    # Raise the torso using just a controller
    rospy.loginfo("Raising torso...")
    torso_action.move_to([0.4, ])

    # Tuck the arm, lower the torso
    grasping_client.tuck()
    torso_action.move_to([0.0, ])


def Face_Recognition_demo():
    # Create a node
    rospy.init_node("face_recognition_demo_node")

    # Make sure sim time is working
    while not rospy.Time.now():
        pass

    # Setup clients
    # Navigation doesnt work very well 
    
    
    
    move_base_pub = MoveBasePublisher()
    torso_action = FollowTrajectoryClient("torso_controller", ["torso_lift_joint"])
    head_action = PointHeadClient()
    grasping_client = GraspingClient()
    
    rospy.loginfo("Start Sequence complete")
    
    rospy.loginfo("Move to position...")
    move_base_pub.move_forwards_backwards(speed_ms=0.3,position_x=1.0, allowed_error=0.1)
    
    # Raise the torso using just a controller
    rospy.loginfo("Raising torso...")
    torso_action.move_to([0.4, ])
    
    # Point the head at the cube we want to pick
    #rospy.loginfo("1,0,-1")
    #head_action.look_at(0.0, 0.0, 2.0, "map")
    

if __name__ == "__main__":
    Face_Recognition_demo()



<p style="background:green;color:white;">**END move_fetch_robot_client.py**</p>

<p style="background:#EE9023;color:white;">**Exercise U5-1**</p>

* Move the Fetch robot close to the people and raise its torso, using the following command.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun face_recognition_tc move_fetch_robot_client.py

* Execute the **recognise_face.py** script and check how it works. Rotate the Fetch robot so that it can try to recognise all 3 people in the scene.

* Once you have checked that it only works with one person (which is called **standing person**), try to also be able to recognise the other two people. This means, taking a picture, and also modifying what you think has to be changed so that it now recognises all 3 people.

<p style="background:green;color:white;">**Data for Exercise U5-1**</p>

* Remember to launch the Graphical Interface window in order to visualize the detections.

<img src="img/font-awesome_desktop.png" width="30" /> 

* When trying to recognise the faces, you will see that only one of them is recognised.

<div style="text-align: left;">
<img src="img/recognise1.png" width="250" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/recognise2.png" width="250" style="float: left; margin: 0px 0px 15px 60px;"/>
<img src="img/recognise3.png" width="250" style="float: left; margin: 0px 0px 15px 60px;"/>
</div>

When a face is successfulyy recognised, you will get a message like this in the output:

In [ ]:
[INFO] [WallTime: 1517396015.764194] [432.655000] MATCH

When face is not correctly recognised, you will get this instead:

In [ ]:
[WARN] [WallTime: 1517397392.869854] [606.444000] NO Match

<p style="background:green;color:white;">**END Data for Exercise U5-1**</p>

<p style="background:#EE9023;color:white;">**END Exercise U5-1**</p>

## Multiple Face Detection at the Same Time

<p style="background:#EE9023;color:white;">**Exercise U5-2**</p>

Do the following:<br>

* Create a new script that is able to recognise all three people at the same time.
* Use the below images for doing the face recognition. You can store them in the **person_img** folder in your package **my_face_recogniser**.


Bob:

<table style="width:100%">
  <tr>
    <th>
    <figure>
      <img id="fig-A.1" src="img/perception_unit4_bobeye.png" width="300"/>
       <center> <figcaption><h2>Bob</h2></figcaption></center>
    </figure>

    </th>
  </tr>
</table>

Naoko:

<table style="width:100%">
  <tr>
    <th>
        <figure>
      <img id="fig-A.2" src="img/perception_unit4_naoko.png" width="300"/>
       <center> <figcaption><h2>Naoko</h2></figcaption></center>
    </figure>
    </th>
  </tr>
</table>

Standing person:

<table style="width:100%">
  <tr>
        
    <th>
        <figure>
      <img id="fig-A.2" src="img/perception_unit5_standing_person2.png" width="300"/>
       <center> <figcaption><h2>Standing Person</h2></figcaption></center>
    </figure>
    </th> 

  </tr>
</table>

<p style="background:#EE9023;color:white;">**END Exercise U5-2**</p>

<p style="background:green;color:white;">**Solution Exercise U5.2**</p>

Please try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

You should have something similar to this:

<img src="img/perception_unit5_recogthreepoepple.png"/>

Of course, an interesting application is to be able to recognise all of these people at the same time. Here you have an example of how it could be done:

<p style="background:green;color:white;">**recognise_multiple_faces.py**</p>

In [ ]:
#!/usr/bin/env python
import os
import rospy
import cv2
import face_recognition
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
import rospkg



class FaceRecogniser(object):

    def __init__(self):
        rospy.loginfo("Start FaceRecogniser Init process...")
        # get an instance of RosPack with the default search paths
        rospack = rospkg.RosPack()
        # get the file path for face_recognition_pkg
        self.path_to_package = rospack.get_path('face_recognition_pkg')
    
        self.bridge_object = CvBridge()
        rospy.loginfo("Start camera suscriber...")
        self.image_sub = rospy.Subscriber("/head_camera/rgb/image_raw",Image,self.camera_callback)
        rospy.loginfo("Finished FaceRecogniser Init process...Ready")
    
    def camera_callback(self,data):
        
        self.recognise(data)

    def recognise(self,data):
        
        # Get a reference to webcam #0 (the default one)
        try:
            # We select bgr8 because its the OpneCV encoding by default
            video_capture = self.bridge_object.imgmsg_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)
        
        
        # Load a sample picture of each person you want to recognise.
        bobeye_image_file = os.path.join(self.path_to_package,"person_img/bobeye.png")
        naoko_image_file = os.path.join(self.path_to_package,"person_img/naoko.png")
        standing_person_image_file = os.path.join(self.path_to_package,"person_img/standing_person.png")
        
        # Get the face encodings for each face in each image file
        # Since there could be more than one face in each image, it returns a list of encordings.
        # But since I know each image only has one face, I only care about the first encoding in each image, so I grab index 0.
        bobeye_image = face_recognition.load_image_file(bobeye_image_file)
        bobeye_face_encoding = face_recognition.face_encodings(bobeye_image)[0]
        
        naoko_image = face_recognition.load_image_file(naoko_image_file)
        naoko_face_encoding = face_recognition.face_encodings(naoko_image)[0]
        
        standing_person_image = face_recognition.load_image_file(standing_person_image_file)
        standing_person_face_encoding = face_recognition.face_encodings(standing_person_image)[0]
        
        
        known_faces = [
                        bobeye_face_encoding,
                        naoko_face_encoding,
                        standing_person_face_encoding
                        ]
        
        # Initialize some variables
        face_locations = []
        face_encodings = []
        face_names = []
        process_this_frame = True


        # Grab a single frame of video
        #ret, frame = video_capture.read()
        
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(video_capture, (0, 0), fx=0.5, fy=0.5)
        
        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(small_frame)
            face_encodings = face_recognition.face_encodings(small_frame, face_locations)
    
            if not face_encodings:
                rospy.logwarn("No Faces found, please get closer...")
    
            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                match = face_recognition.compare_faces(known_faces, face_encoding)
                name = "Unknown"
    
                if match[0] and not match[1] and not match[2]:
                    name = "Bob"
                elif not match[0] and match[1] and not match[2]:
                    name = "Naoko"
                elif not match[0] and not match[1] and match[2]:
                    name = "Standing_person"
                elif not match[0] and not match[1] and not match[2]:
                    name = ""
                else:
                    name = "SOMETHING_IS_WRONG"
                rospy.loginfo(name)
    
                face_names.append(name)
    
        process_this_frame = not process_this_frame
    
        
        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 2
            right *= 2
            bottom *= 2
            left *= 2
    
            # Draw a box around the face
            cv2.rectangle(video_capture, (left, top), (right, bottom), (0, 0, 255), 2)
    
            # Draw a label with a name below the face
            cv2.rectangle(video_capture, (left, bottom - 35), (right, bottom), (0, 0, 255))
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(video_capture, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            
        
        # Display the resulting image
        cv2.imshow("Image window", video_capture)
        cv2.waitKey(1)
        



def main():
    rospy.init_node('face_recognising_python_node', anonymous=True)
   
    line_follower_object = FaceRecogniser()

    rospy.spin()
    cv2.destroyAllWindows()

    
if __name__ == '__main__':
    main()

<p style="background:green;color:white;">**END recognise_multiple_faces.py**</p>

<p style="background:#EE9023;color:white;">**Exercise EXTRA U5-3**</p>

Now, create a program that does the following:<br>

* Reads a topic where someone publishes the name of one of the three people.
* Uses the Face Detection system created in the previous unit to look around for people.
* Once the robot has detected a face, it moves towards it until it recognises which face it is. If it's the person that it's looking for, the program ends. If not, it memorises that person's location to know it's not the person it's looking for. Then, it starts the search again, filtering the people already discarded.
* Publishes the face data so that you can use the <a href="http://jsk-visualization.readthedocs.io/en/latest/jsk_rviz_plugins/index.html">FaceDetection RVIZ elements</a> to see the recognised faces. Use these markers to also be able to print the name of the person recognised. If that person has been discarded, then the marker has to be of a different color, or indicate it in some other way.


<p style="background:#EE9023;color:white;">**END Exercise EXTRA U5-3**</p>

<p style="background:#417FB1;color:white;">**Project**</p>

You can now do the fifth exercise of the Aibo Project. There, you will have to look for your owner, Olive. There will be two humans in the scene. You will have to find Olive and give her paw.

<p style="background:#417FB1;color:white;">**END Project**</p>